In [3]:
import json
from deepdiff import DeepDiff
from pprint import pprint

stg_data_path = "../temp_file/stg_product.jsons"
prod_data_path = "../temp_file/prod_product.jsons"

def convert_json(file_path):
    with open (file_path, "r") as myfile:
        data=myfile.read().replace("}\n{", "},\n{")
        data = "[" + data + "]"
        data = json.loads(data)
        return data

# write missing products to csv

# all_rids = []  
# with open(output_filename, "w") as output:
#     writer = csv.writer(output, lineterminator='\n')
#     for val in all_rids:
#         writer.writerow([val])
    
def compare_products(stg_data, prod_data):
    print("Comparing Data")
#     incoming data as dictionary

def main():
    new_stg_data = convert_json(stg_data_path)
    new_prod_data = convert_json(prod_data_path)
    
main()
    

[{'externalID': '1047237787684', 'active': True, 'enabled': True, 'attributes': {'productPageUrl': {'default': 'http://google.com', 'en_US': 'https://shop-cst-tires.myshopify.com/products/roly-poly-1'}, 'imageUrl': {'default': 'https://cdn.shopify.com/s/files/1/2614/6598/products/C1936-Roly-Poly-Merged.jpg?v=1547649699'}, 'INVALID_UPC': ['TB72666100'], 'name': {'default': 'RED GREEN BLUE', 'en_US': 'RED GREEN BLUE'}, 'upcs': []}, 'families': {'workbench_expanded': [], 'workbench': [], 'feed': ['TB72666100'], 'feed_expanded': ['BV_FE_FAMILY:TB72666100'], 'workbench_expanded_deleted': [], 'workbench_deleted': [], 'feed_deleted': [], 'feed_expanded_deleted': []}, 'categoryExternalID': 'BV_MISCELLANEOUS_CATEGORY', 'brandExternalID': 'ecov7ii6u1exmd4bhu6hntp85', 'created': 1553935148960, 'lastUpdated': 1553935148960, 'lastSeen': 1553935149752, 'clientName': 'csttires'}, {'externalID': '1047237820452', 'active': True, 'enabled': True, 'attributes': {'productPageUrl': {'default': 'https://sho